# MÔ HÌNH: DỰ ĐOÁN KHÁCH HÀNG CHURN (CUSTOMER CHURN PREDICTION)

## Mục tiêu bài toán
Dự đoán khách hàng có khả năng "không quay lại mua" (churn) hay không dựa trên RFM và lịch sử mua hàng để:
- Chiến dịch giữ chân khách hàng (retention campaigns)
- Ưu tiên chăm sóc khách hàng có nguy cơ churn cao
- Phân tích nguyên nhân churn
- Tối ưu hóa marketing spend

## Input (Dữ liệu)
- **File nguồn:** `data/merged_supply_weather_clean.parquet`
- **Aggregation:** Group by `Order Customer Id`
- **Các nhóm feature:** RFM, Customer History, Engagement, Location

## Output (Yêu cầu dự đoán)
- **Target:** `churn` (binary: 0 = active, 1 = churned)
- **Định nghĩa churn:** Recency > 180 days (6 tháng)
- **Output:** Xác suất churn, label, và recommendations

## Thông tin phiên bản
- **Ngày:** 2024
- **Phiên bản:** 1.0
- **Dataset:** merged_supply_weather_clean.parquet


In [ ]:
# Import thư viện & cấu hình chung
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    roc_curve, f1_score, accuracy_score, precision_score, recall_score
)
import xgboost as xgb
from imblearn.over_sampling import SMOTE
import os
import warnings
warnings.filterwarnings('ignore')

# Cấu hình matplotlib
%matplotlib inline
plt.style.use('default')
sns.set_palette("husl")

# Random seed để đảm bảo reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ Đã import thư viện và cấu hình xong")


## 1. Load dữ liệu

Load dữ liệu merged đã chuẩn hóa từ file `data/merged_supply_weather_clean.parquet`.


In [ ]:
# Load dữ liệu merged đã chuẩn hóa
data_path = os.path.join('..', 'data', 'merged_supply_weather_clean.parquet')

if not os.path.exists(data_path):
    raise FileNotFoundError(f"File không tìm thấy: {data_path}\nVui lòng chạy: python scripts/merge_supplychain_weather.py trước")

df = pd.read_parquet(data_path)
print(f"✓ Đã load {len(df):,} records")
print(f"✓ Số cột: {len(df.columns)}")

# Hiển thị thông tin cơ bản
print("\n=== THÔNG TIN DATASET ===")
df.info()

print("\n=== 5 DÒNG ĐẦU TIÊN ===")
df.head()


## 2. Tính toán RFM & Customer Features

Tính toán RFM (Recency, Frequency, Monetary) và các features khác cho từng khách hàng.


In [ ]:
# Define snapshot date (last date in dataset)
snapshot_date = df['order date (DateOrders)'].max()
print(f"Snapshot date: {snapshot_date}")

# Calculate RFM for each customer
df['order_date'] = pd.to_datetime(df['order date (DateOrders)'])

# Recency: Days since last order
last_order = df.groupby('Order Customer Id')['order_date'].max()
recency = (snapshot_date - last_order).dt.days

# Frequency: Number of orders
frequency = df.groupby('Order Customer Id')['Order Id'].nunique()

# Monetary: Total sales
monetary = df.groupby('Order Customer Id')['Sales'].sum()

# Combine RFM
customer_features = pd.DataFrame({
    'customer_id': recency.index,
    'rfm_recency': recency.values,
    'rfm_frequency': frequency.values,
    'rfm_monetary': monetary.values
})

print(f"✓ Đã tính RFM cho {len(customer_features):,} customers")


In [ ]:
# Additional customer features
customer_stats = df.groupby('Order Customer Id').agg({
    'Sales': ['sum', 'mean', 'std'],
    'Order Id': 'nunique',
    'Category Name': 'nunique',  # Category diversity
    'Order Country': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown',  # Most common country
    'order_date': ['min', 'max']  # First and last order dates
}).reset_index()

customer_stats.columns = [
    'customer_id', 'total_sales', 'avg_order_value', 'std_order_value',
    'total_orders', 'category_diversity', 'preferred_country',
    'first_order_date', 'last_order_date'
]

# Merge
customer_features = customer_features.merge(customer_stats, on='customer_id', how='left')

# Days since first order
customer_features['days_since_first_order'] = (
    snapshot_date - pd.to_datetime(customer_features['first_order_date'])
).dt.days

# Average discount
customer_discount = df.groupby('Order Customer Id')['Order Item Discount'].mean()
customer_features = customer_features.merge(
    customer_discount.rename('avg_discount'),
    left_on='customer_id',
    right_index=True,
    how='left'
)

print(f"✓ Đã tính thêm customer features")


## 3. Định nghĩa Churn Label

**Churn definition:** Recency > 180 days (6 tháng) - Khách hàng không mua lại trong 6 tháng gần nhất được coi là churn.


In [ ]:
# Churn definition: Recency > 180 days (6 months)
churn_threshold = 180
customer_features['churn'] = (customer_features['rfm_recency'] > churn_threshold).astype(int)

churn_rate = customer_features['churn'].mean()
print(f"Churn definition: Recency > {churn_threshold} days")
print(f"Tỉ lệ churn: {churn_rate*100:.2f}%")
print(f"Churned customers: {customer_features['churn'].sum():,}")
print(f"Active customers: {(customer_features['churn']==0).sum():,}")

# Visualize churn distribution
plt.figure(figsize=(8, 5))
customer_features['churn'].value_counts().plot(kind='bar', color=['green', 'red'])
plt.title('Phân phối Churn', fontsize=14, fontweight='bold')
plt.xlabel('Churn (0=Active, 1=Churned)', fontsize=12)
plt.ylabel('Số lượng', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


## 4. EDA & Trực quan hóa

Phân tích mối tương quan giữa RFM và churn.


In [ ]:
# Phân tích RFM vs Churn
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Recency
customer_features.boxplot(column='rfm_recency', by='churn', ax=axes[0])
axes[0].set_title('Phân phối Recency theo Churn', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Churn', fontsize=11)
axes[0].set_ylabel('Recency (ngày)', fontsize=11)
plt.suptitle('')

# Frequency
customer_features.boxplot(column='rfm_frequency', by='churn', ax=axes[1])
axes[1].set_title('Phân phối Frequency theo Churn', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Churn', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
plt.suptitle('')

# Monetary
customer_features.boxplot(column='rfm_monetary', by='churn', ax=axes[2])
axes[2].set_title('Phân phối Monetary theo Churn', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Churn', fontsize=11)
axes[2].set_ylabel('Monetary ($)', fontsize=11)
plt.suptitle('')

plt.tight_layout()
plt.show()


## 5. Tiền xử lý & Feature Engineering

**Pipeline xử lý:**
- Chọn features: RFM, Customer History, Engagement, Location
- Xử lý missing values: Fill 0 cho numeric
- Encoding: One-hot encoding cho country (top 10)
- Xử lý class imbalance: SMOTE


In [ ]:
# Chọn features
feature_cols = [
    'rfm_recency', 'rfm_frequency', 'rfm_monetary',
    'total_sales', 'avg_order_value', 'std_order_value',
    'total_orders', 'category_diversity',
    'days_since_first_order', 'avg_discount'
]

# Handle missing values
customer_features[feature_cols] = customer_features[feature_cols].fillna(0)

# Country encoding (one-hot for top countries)
top_countries = customer_features['preferred_country'].value_counts().head(10).index
for country in top_countries:
    customer_features[f'country_{country}'] = (
        customer_features['preferred_country'] == country
    ).astype(int)

# Prepare feature matrix
X = customer_features[feature_cols + [f'country_{c}' for c in top_countries]].copy()
y = customer_features['churn'].copy()

print(f"Feature matrix shape: {X.shape}")
print(f"Features: {list(X.columns)}")


## 6. Chia tập train/test

**Tiêu chí chia:** Time-based split (80% train, 20% test) theo `last_order_date` để tránh data leakage.
- Train: 80% customers (theo last_order_date)
- Test: 20% customers


In [ ]:
# Time-based split
customer_features_sorted = customer_features.sort_values('last_order_date')
split_idx = int(len(customer_features_sorted) * 0.8)

train_mask = customer_features_sorted.index[:split_idx]
test_mask = customer_features_sorted.index[split_idx:]

X_train = X.loc[train_mask]
X_test = X.loc[test_mask]
y_train = y.loc[train_mask]
y_test = y.loc[test_mask]

print(f"Train set: {len(X_train):,} samples (churn rate: {y_train.mean()*100:.2f}%)")
print(f"Test set: {len(X_test):,} samples (churn rate: {y_test.mean()*100:.2f}%)")


In [ ]:
# Xử lý class imbalance với SMOTE
smote = SMOTE(random_state=RANDOM_STATE)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"Trước SMOTE: {len(X_train):,} samples (churn: {y_train.sum():,})")
print(f"Sau SMOTE: {len(X_train_balanced):,} samples (churn: {y_train_balanced.sum():,})")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test)

print("✓ Đã scale features")


## 7. Huấn luyện mô hình

**Các mô hình sẽ thử:**
- **Baseline:** Logistic Regression (đơn giản, dễ interpret)
- **Tree-based:** Random Forest, XGBoost (xử lý non-linear, feature importance)


In [ ]:
# 7.1. Logistic Regression (Baseline)
lr_model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
lr_model.fit(X_train_scaled, y_train_balanced)
y_pred_lr = lr_model.predict(X_test_scaled)
y_pred_proba_lr = lr_model.predict_proba(X_test_scaled)[:, 1]

print("=== KẾT QUẢ LOGISTIC REGRESSION ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_lr):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_lr):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_lr):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))


In [ ]:
# 7.2. Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=RANDOM_STATE,
    class_weight='balanced',
    n_jobs=-1
)
rf_model.fit(X_train_balanced, y_train_balanced)
y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

print("=== KẾT QUẢ RANDOM FOREST ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_rf):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba_rf):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))


In [ ]:
# 7.3. XGBoost
scale_pos_weight = (y_train_balanced == 0).sum() / (y_train_balanced == 1).sum()

xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=RANDOM_STATE,
    scale_pos_weight=scale_pos_weight
)
xgb_model.fit(X_train_balanced, y_train_balanced)
y_pred_xgb = xgb_model.predict(X_test)
y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

print("=== KẾT QUẢ XGBOOST ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_xgb):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba_xgb):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_xgb):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_xgb):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb))


## 8. Đánh giá mô hình & Trực quan hóa

**Metrics chính:**
- **Accuracy:** Tỉ lệ dự đoán đúng
- **F1 Score:** Harmonic mean của Precision và Recall
- **AUC-ROC:** Diện tích dưới đường ROC
- **Precision@TopK:** Precision trong top K khách hàng có risk cao nhất (quan trọng cho business)


In [ ]:
# So sánh các mô hình
models_dict = {
    'Logistic Regression': (y_pred_proba_lr, y_pred_lr),
    'Random Forest': (y_pred_proba_rf, y_pred_rf),
    'XGBoost': (y_pred_proba_xgb, y_pred_xgb)
}

results = []
for name, (proba, pred) in models_dict.items():
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, pred),
        'F1': f1_score(y_test, pred),
        'AUC-ROC': roc_auc_score(y_test, proba),
        'Precision': precision_score(y_test, pred),
        'Recall': recall_score(y_test, pred)
    })

results_df = pd.DataFrame(results)
print("\n=== SO SÁNH CÁC MÔ HÌNH ===")
print(results_df.to_string(index=False))


In [ ]:
# Precision@TopK (Top K customers with highest churn risk)
K = 1000
for name in models_dict.keys():
    proba = models_dict[name][0]
    top_k_indices = np.argsort(proba)[-K:][::-1]
    top_k_actual = y_test.iloc[top_k_indices]
    precision_at_k = top_k_actual.sum() / K
    print(f"\n{name} - Precision@Top{K}: {precision_at_k:.4f}")


In [ ]:
# ROC Curves
plt.figure(figsize=(10, 8))

for name in models_dict.keys():
    proba = models_dict[name][0]
    fpr, tpr, _ = roc_curve(y_test, proba)
    auc = roc_auc_score(y_test, proba)
    plt.plot(fpr, tpr, label=f'{name} (AUC={auc:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random', linestyle='--')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves - So sánh các mô hình', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Confusion Matrices
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for idx, name in enumerate(models_dict.keys()):
    pred = models_dict[name][1]
    cm = confusion_matrix(y_test, pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx], cbar_kws={'label': 'Count'})
    axes[idx].set_title(f'{name}\nAccuracy: {accuracy_score(y_test, pred):.3f}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Predicted', fontsize=11)
    axes[idx].set_ylabel('Actual', fontsize=11)

plt.tight_layout()
plt.show()


In [ ]:
# Feature Importance (XGBoost)
if hasattr(xgb_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False).head(15)
    
    plt.figure(figsize=(10, 8))
    sns.barplot(data=feature_importance, y='feature', x='importance', palette='viridis')
    plt.title('Top 15 Feature Importances (XGBoost)', fontsize=14, fontweight='bold')
    plt.xlabel('Importance', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("\n=== TOP 10 FEATURES QUAN TRỌNG NHẤT ===")
    print(feature_importance.head(10).to_string(index=False))


## 9. Kết luận & Gợi ý

### Kết quả chính
- **Model tốt nhất:** XGBoost (dựa trên AUC-ROC và F1 Score)
- **Metric chính:** AUC-ROC, F1 Score, Precision@TopK (quan trọng cho business)

### Nhận xét
**Features quan trọng (dựa trên feature importance):**
- `rfm_recency`: Số ngày từ lần mua cuối là yếu tố quan trọng nhất (quyết định churn)
- `rfm_frequency`: Số đơn hàng
- `rfm_monetary`: Tổng giá trị mua hàng
- `days_since_first_order`: Tuổi khách hàng
- `avg_order_value`: Giá trị đơn trung bình

### Hạn chế
- ⚠️ Churn definition (180 days) có thể cần điều chỉnh theo business
- ⚠️ Thiếu behavioral features (website visits, email opens, customer support interactions)
- ⚠️ Thiếu external data (competitor pricing, market conditions)
- ⚠️ Model chưa được hyperparameter tuning kỹ

### Hướng phát triển
1. **Feature Engineering:**
   - Customer lifetime value (CLV)
   - Purchase velocity (tốc độ mua)
   - Product return rate
   - Customer support interactions
   - Website/app engagement metrics

2. **Model Improvement:**
   - Hyperparameter tuning (GridSearchCV/RandomSearchCV)
   - Ensemble methods (voting, stacking)
   - Deep Learning (nếu có đủ dữ liệu)

3. **Business Logic:**
   - Điều chỉnh churn definition theo business
   - Segment-specific models (ví dụ: B2B vs B2C)
   - Cohort analysis

4. **Deployment:**
   - Real-time prediction API
   - Model monitoring (drift detection)
   - Retrain định kỳ
   - Integration với CRM system
